In [ ]:
import sys
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy.misc import imresize
%matplotlib inline
from IPython import display
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing import image

In [3]:
# Initialising the CNN
classifier = Sequential()

# Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
# Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.25))
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.25))
# Flattening
classifier.add(Flatten())

# Full connection
classifier.add(Dense(units = 64, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 5, activation='softmax'))
# Compiling the CNN
classifier.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [4]:
# Fitting the CNN to the images
train_datagen = image.ImageDataGenerator(rescale=1./255,
                                   rotation_range=180,
                                   width_shift_range=0.3,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
test_datagen = image.ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                  target_size = (64, 64),
                                                  batch_size = 32,
                                                  class_mode = 'categorical')
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                             target_size = (64, 64),
                                             batch_size = 32,
                                             class_mode = 'categorical')
classifier.fit_generator(training_set,
                         steps_per_epoch = 250,
                         epochs = 5,
                         validation_data = test_set,
                         validation_steps = 500,
                         verbose = 2)
classifier.save('cnn_model.h5')

Found 36 images belonging to 5 classes.
Found 18 images belonging to 5 classes.
Epoch 1/5
 - 856s - loss: 1.1224 - acc: 0.5475 - val_loss: 0.9624 - val_acc: 0.7222
Epoch 2/5
 - 825s - loss: 0.5616 - acc: 0.8159 - val_loss: 0.7119 - val_acc: 0.7222
Epoch 3/5
 - 897s - loss: 0.4046 - acc: 0.8731 - val_loss: 0.9511 - val_acc: 0.7778
Epoch 4/5
 - 877s - loss: 0.3480 - acc: 0.8998 - val_loss: 1.0336 - val_acc: 0.7778
Epoch 5/5
 - 817s - loss: 0.2725 - acc: 0.9203 - val_loss: 1.2240 - val_acc: 0.7222


ImportError: `save_model` requires h5py.

In [15]:
classifier.save_weights('cnn_weights.h5')

ImportError: `save_weights` requires h5py.

In [ ]:
# Making new predictions
count = 0
for i in range(1, 15):
    test_image = image.load_img('dataset/single_prediction/cat_or_dog_{}.jpg'.format(i), target_size = (64, 64))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image, axis = 0)
    result = classifier.predict(test_image)
    training_set.class_indices
    print(result)
    if result[0][0] == 1:
        count += 1
        prediction = 'dog'
    else:
        prediction = 'cat'
    plt.title(prediction)
    plt.imshow(test_image[0].astype(float))
    plt.show()
print(count/14)

In [ ]:
# Save the weights
classifier.save_weights('weights.h5')

# Save the model architecture
with open('catdog_architecture.json', 'w') as f:
    f.write(classifier.to_json())

In [16]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('catdog_architecture.json', 'r') as f:
    model = model_from_json(f.read())

# Load weights into the new model
model.load_weights('catdog_weights.h5')

ImportError: `load_weights` requires h5py.

In [ ]:
def backproject(source, target, levels = 2, scale = 1):
    hsv = cv2.cvtColor(source,	cv2.COLOR_BGR2HSV)
    hsvt = cv2.cvtColor(target,	cv2.COLOR_BGR2HSV)
    # calculating object histogram
    roihist = cv2.calcHist([hsv],[0, 1], None, [levels, levels], [0, 180, 0, 256] )

    # normalize histogram and apply backprojection
    cv2.normalize(roihist,roihist,0,255,cv2.NORM_MINMAX)
    dst = cv2.calcBackProject([hsvt],[0,1],roihist,[0,180,0,256], scale)
    return dst

def saliency_by_backprojection(img):
    cv2.pyrMeanShiftFiltering(img, 2, 10, img, 4)

    backproj = np.uint8(backproject(img, img, levels = 2))
    cv2.normalize(backproj,backproj,0,255,cv2.NORM_MINMAX)
    saliencies = [backproj, backproj, backproj]
    saliency = cv2.merge(saliencies)

    cv2.pyrMeanShiftFiltering(saliency, 20, 200, saliency, 2)
    saliency = cv2.cvtColor(saliency, cv2.COLOR_BGR2GRAY)
    cv2.equalizeHist(saliency, saliency)

    return 255-saliency

def saliency_map(img):
    saliency_hsv = saliency_by_backprojection(img * 1)
    saliency = saliency_hsv
    (T, saliency) = cv2.threshold(saliency, 200, 255, cv2.THRESH_BINARY)
    return saliency

def largest_contours_rect(saliency):
    _, contours, hierarchy = cv2.findContours(saliency * 1,cv2.RETR_LIST,cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key = cv2.contourArea)
    return cv2.boundingRect(contours[-1])

def refine_saliency_with_grabcut(img, saliency):
    rect = largest_contours_rect(saliency)
    bgdmodel = np.zeros((1, 65),np.float64)
    fgdmodel = np.zeros((1, 65),np.float64)
    saliency[np.where(saliency > 0)] = cv2.GC_FGD
    mask = saliency
    try:
        cv2.grabCut(img, mask, rect, bgdmodel, fgdmodel, 1, cv2.GC_INIT_WITH_RECT)
    except:
        pass
    mask = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    return mask

def backprojection_saliency(img):
    saliency = saliency_map(img)
    mask = refine_saliency_with_grabcut(img, saliency)
    return mask

def bounding_box(mask):
    obj = np.where(mask != 0)
    bbox = np.min(obj[0]), np.max(obj[0]), np.min(obj[1]), np.max(obj[1])
    return bbox

In [ ]:
img = cv2.imread("flower_original.jpg", 1)
img = cv2.resize(img, (640, 480))
mask = backprojection_saliency(img)
segmentation = img*mask[:,:,np.newaxis]
plt.imshow(mask)
# plt.imshow(segmentation)

In [ ]:
print(np.count_nonzero(mask), mask.size)

In [ ]:
%matplotlib inline
%matplotlib inline
vc = cv2.VideoCapture(0)

if vc.isOpened(): # try to get the first frame
    is_capturing, frame = vc.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
    webcam_preview = plt.imshow(frame)   
else:
    is_capturing = False

while is_capturing:
    try:    # Lookout for a keyboardInterrupt to stop the script
        is_capturing, frame = vc.read()
        frame_big = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    # makes the blues image look real colored
        frame = imresize(frame_big, (180,320))
        mask = backprojection_saliency(frame)
        segmentation = frame*mask[:,:,np.newaxis]
        bbox = bounding_box(mask)
        frame[bbox[0]:bbox[1],bbox[2]:bbox[3],1] = np.clip(2 * frame[bbox[0]:bbox[1],bbox[2]:bbox[3],1], None, 255)
        webcam_preview.set_data(frame)
        plt.draw()
        display.clear_output(wait=True)
        display.display(plt.gcf())

        plt.pause(1)    # the pause time is = 1 / framerate
    except KeyboardInterrupt:
        vc.release()
    except:
        continue